In [1]:
import pandas as pd
import numpy as np
import os
import sys
import talib
import joblib
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
warnings.filterwarnings('ignore')

In [5]:
from binance.client import Client
sys.path.append(os.path.abspath(".."))  # root /PycharmProjects/MMAT
from config.load_env import load_keys

keys = load_keys()
#print("Loaded keys:", keys)
client = Client(keys['api_key'], keys['secret_key'])

In [6]:
# ================== 增强的特征工程 ==================
class AdvancedFeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self, lookback_periods=[5, 10, 20, 50]):
        self.lookback_periods = lookback_periods
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, df):
        # 确保索引是时间序列
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        
        # 基础价格特征
        df['price_change'] = df['close'].pct_change()
        df['high_low_ratio'] = (df['high'] - df['low']) / df['close']
        df['close_open_ratio'] = (df['close'] - df['open']) / df['open']
        
        # 多时间框架特征
        for period in [15, 30, 60]:  # 15min, 30min, 1h
            df[f'ema_{period}'] = talib.EMA(df['close'], timeperiod=period)
            df[f'sma_{period}'] = talib.SMA(df['close'], timeperiod=period)
            df[f'ma_cross_{period}'] = np.where(df[f'ema_{period}'] > df[f'sma_{period}'], 1, -1)
        
        # 波动率特征
        df['atr'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
        df['natr'] = talib.NATR(df['high'], df['low'], df['close'], timeperiod=14)
        df['volatility'] = df['close'].rolling(20).std() / df['close'].rolling(20).mean()
        
        # 成交量特征
        df['volume_ma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / df['volume_ma']
        df['obv'] = talib.OBV(df['close'], df['volume'])
        
        # 动量特征
        df['rsi'] = talib.RSI(df['close'], timeperiod=14)
        df['macd'], df['macd_signal'], _ = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
        df['macd_hist'] = df['macd'] - df['macd_signal']
        df['stoch_k'], df['stoch_d'] = talib.STOCH(df['high'], df['low'], df['close'], fastk_period=14, slowk_period=3, slowd_period=3)
        
        # 高级特征
        df['adx'] = talib.ADX(df['high'], df['low'], df['close'], timeperiod=14)
        df['cci'] = talib.CCI(df['high'], df['low'], df['close'], timeperiod=20)
        df['mfi'] = talib.MFI(df['high'], df['low'], df['close'], df['volume'], timeperiod=14)
        
        # 价格形态特征
        df['inside_bar'] = ((df['high'] < df['high'].shift(1)) & (df['low'] > df['low'].shift(1))).astype(int)
        df['outside_bar'] = ((df['high'] > df['high'].shift(1)) & (df['low'] < df['low'].shift(1))).astype(int)
        
        # 时间特征
        df['hour'] = df.index.hour
        df['day_of_week'] = df.index.dayofweek
        df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
        
        # 滞后特征
        for lag in [1, 2, 3, 5]:
            df[f'return_lag{lag}'] = df['price_change'].shift(lag)
            df[f'volume_ratio_lag{lag}'] = df['volume_ratio'].shift(lag)
        
        # 目标编码特征（需要小心处理避免前瞻偏差）
        # 这里省略，实际应用中需要谨慎实现
        
        # 删除NaN值
        df = df.dropna()
        
        return df

# ================== 元模型架构 ==================
class MetaModel:
    def __init__(self):
        self.base_models = [
            ('xgb_base', XGBClassifier(
                n_estimators=300,
                max_depth=5,
                learning_rate=0.05,
                subsample=0.8,
                colsample_bytree=0.8,
                gamma=0.1,
                reg_alpha=0.1,
                reg_lambda=0.1,
                objective='multi:softprob',
                num_class=3,
                use_label_encoder=False,
                eval_metric='mlogloss'
            )),
            ('rf', RandomForestClassifier(
                n_estimators=200,
                max_depth=7,
                class_weight='balanced',
                random_state=42
            )),
            ('gbm', GradientBoostingClassifier(
                n_estimators=150,
                learning_rate=0.05,
                max_depth=4,
                random_state=42
            ))
        ]
        
        self.meta_model = XGBClassifier(
            n_estimators=100,
            max_depth=3,
            learning_rate=0.1,
            objective='multi:softprob',
            num_class=3,
            use_label_encoder=False,
            eval_metric='mlogloss'
        )
        self.scaler = StandardScaler()

    def fit(self, X, y):
        sample_weights = compute_sample_weight(class_weight='balanced', y=y)
        base_preds = []
        for name, model in self.base_models:
            model.fit(X, y, sample_weight=sample_weights)
            preds = model.predict_proba(X)
            base_preds.append(preds)

        meta_X = np.hstack(base_preds)
        meta_X_scaled = self.scaler.fit_transform(meta_X)

        self.meta_model.fit(meta_X_scaled, y, sample_weight=sample_weights)
        self.base_models = [(name, model) for name, model in self.base_models]


    def predict_proba(self, X):
        base_preds = [model.predict_proba(X) for _, model in self.base_models]
        meta_X = np.hstack(base_preds)
        meta_X_scaled = self.scaler.transform(meta_X)
        return self.meta_model.predict_proba(meta_X_scaled)

    def predict(self, X):
        proba = self.predict_proba(X)
        # 手动调整惩罚，鼓励预测为 -1, 1
        adjusted = proba * np.array([1.2, 0.9, 1.2])  # [down, neutral, up]
        return np.argmax(adjusted, axis=1)

def balance_classes(df, label_col='label'):
    from sklearn.utils import resample
    df_neg = df[df[label_col] == -1]
    df_zero = df[df[label_col] == 0]
    df_pos = df[df[label_col] == 1]

    max_len = max(len(df_neg), len(df_zero), len(df_pos))

    df_neg_up = resample(df_neg, replace=True, n_samples=max_len, random_state=42)
    df_zero_up = resample(df_zero, replace=True, n_samples=max_len, random_state=42)
    df_pos_up = resample(df_pos, replace=True, n_samples=max_len, random_state=42)

    df_balanced = pd.concat([df_neg_up, df_zero_up, df_pos_up]).sample(frac=1, random_state=42)
    return df_balanced


In [7]:
%%html
<style>
div.output_scroll {
  height: auto !important;
  max-height: none !important;
}
</style>


In [8]:
import os
import webbrowser
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

def plot_pattern_results(df, patterns, symbol, max_points=2000, buffer=50, open_browser=True):
    """
    Plots candlestick charts with pattern signals, ensuring no future leakage and index alignment.

    Parameters:
    - df: DataFrame with signals already generated
    - patterns: list of pattern names (e.g. ['BullishEngulfing', 'ThreeLineStrike'])
    - symbol: str, name of the instrument
    - max_points: how many points to show in the plot (excluding buffer)
    - buffer: number of extra candles before the plotted region (e.g. to show full 3-line strikes)
    - open_browser: whether to open HTML chart in browser
    """

    # Determine the starting point with buffer
    start_index = max(len(df) - max_points - buffer, 0)
    df_plot = df.iloc[start_index:].copy()

    output_dir = './plots/'
    os.makedirs(output_dir, exist_ok=True)

    top_patterns = ['InvertedHammer', 'ThreeLineStrike', 'HangingMan'] # Can adjust - add other candlestick pattern

    for name in top_patterns:
        signal_col = f'Signal_{name}'

        if signal_col not in df_plot.columns:
            print(f"Warning: Signal column '{signal_col}' not found for {name}. Skipping plot.")
            continue

        up_signals = df_plot[df_plot[signal_col] == 1]
        down_signals = df_plot[df_plot[signal_col] == -1]
        neutral_signals = df_plot[df_plot[signal_col] == 9]

        fig = make_subplots(
            rows=2, cols=1,
            shared_xaxes=True,
            vertical_spacing=0.1,
            subplot_titles=['Candlestick + MA', 'RSI'],
            row_heights=[0.7, 0.3]
        )

        fig.add_trace(
            go.Candlestick(
                x=df_plot.index,
                open=df_plot['open'],
                high=df_plot['high'],
                low=df_plot['low'],
                close=df_plot['close'],
                name='Candlestick',
                increasing_line_color='green',
                decreasing_line_color='red'
            ),
            row=1, col=1
        )

        fig.add_trace(
            go.Scatter(x=df_plot.index, y=df_plot['MA20'], mode='lines', name='20 MA', line=dict(color='blue')),
            row=1, col=1
        )
        fig.add_trace(
            go.Scatter(x=df_plot.index, y=df_plot['MA50'], mode='lines', name='50 MA', line=dict(color='purple')),
            row=1, col=1
        )

        if not up_signals.empty:
            fig.add_trace(
                go.Scatter(
                    x=up_signals.index,
                    y=up_signals['close'] * 1.005,
                    mode='markers',
                    marker=dict(symbol='triangle-up', color='green', size=10),
                    name='Bullish Signal',
                    text=[f'Bullish {name}' for _ in range(len(up_signals))],
                    hoverinfo='text+x+y'
                ),
                row=1, col=1
            )
        if not down_signals.empty:
            fig.add_trace(
                go.Scatter(
                    x=down_signals.index,
                    y=down_signals['close'] * 0.995,
                    mode='markers',
                    marker=dict(symbol='triangle-down', color='red', size=10),
                    name='Bearish Signal',
                    text=[f'Bearish {name}' for _ in range(len(down_signals))],
                    hoverinfo='text+x+y'
                ),
                row=1, col=1
            )
        if not neutral_signals.empty:
            fig.add_trace(
                go.Scatter(
                    x=neutral_signals.index,
                    y=neutral_signals['close'],
                    mode='markers',
                    marker=dict(symbol='circle', color='gray', size=8),
                    name='Neutral Signal',
                    text=[f'Neutral {name}' for _ in range(len(neutral_signals))],
                    hoverinfo='text+x+y'
                ),
                row=1, col=1
            )

        fig.add_trace(
            go.Scatter(x=df_plot.index, y=df_plot['RSI'], mode='lines', name='RSI', line=dict(color='blue')),
            row=2, col=1
        )
        fig.add_hline(y=50, line_dash='dash', line_color='black', row=2, col=1)

        fig.update_layout(
            title=f'{name} Signals for {symbol} with MA and RSI',
            xaxis_title='Time',
            yaxis_title='Price ($)',
            yaxis2_title='RSI',
            xaxis_rangeslider_visible=False,
            showlegend=True,
            height=600,
            template='plotly_white'
        )

        html_path = os.path.join(output_dir, f'{name}_signals.html')
        fig.write_html(html_path)
        print(f"Saved plot for {name} to {html_path}")

        if open_browser:
            abs_path = os.path.abspath(html_path)
            webbrowser.open(f'file://{abs_path}')
            print(f"Opened plot for {name} in default browser")


In [9]:
# ================== signal_prediction_pipeline_updated.py ==================
# 整合量化信号预测 pipeline，包含：
# 1. 分位数标签 (calculate_target_quantile)
# 2. 样本平衡 (SMOTE)
# 3. Block Bootstrap 白检验 (block_bootstrap_pval)
# 4. 权重缩放 (weighted_signal_evaluation)
# 5. 蜡烛图形态特征 (integrate_candlestick_features)

import pandas as pd
import numpy as np
import os
import sys
import talib
import joblib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from imblearn.over_sampling import SMOTE
# from arch.bootstrap import StationaryBootstrap

# === 分位数标签函数 ===
def calculate_target_quantile(df, future_bars=3, lower_q=0.2, upper_q=0.8):
    future_return = df['close'].shift(-future_bars) / df['close'] - 1
    lower = future_return.quantile(lower_q)
    upper = future_return.quantile(upper_q)
    conds = [
        future_return <= lower,
        (future_return > lower) & (future_return < upper),
        future_return >= upper
    ]
    labels = [-1, 0, 1]
    direction = np.select(conds, labels, default=0)
    magnitude = future_return.abs()
    return pd.DataFrame({'direction': direction, 'magnitude': magnitude}, index=df.index)

# === 简单随机上采样替代 SMOTE ===
def simple_oversample(X, y):
    df_train = X.copy()
    df_train['direction'] = y.values
    majority = df_train[df_train['direction'] == 0]
    minority = df_train[df_train['direction'] != 0]
    minority_upsampled = resample(
        minority,
        replace=True,
        n_samples=len(majority),
        random_state=42
    )
    df_bal = pd.concat([majority, minority_upsampled])
    y_bal = df_bal['direction']
    X_bal = df_bal.drop(columns='direction')
    return X_bal, y_bal

# === 蜡烛图形态特征 ===
def calculate_patterns(df):
    funcs = {
        'Hammer': talib.CDLHAMMER,
        'InvertedHammer': talib.CDLINVERTEDHAMMER,
        'BullishEngulfing': lambda o,h,l,c: np.where(talib.CDLENGULFING(o,h,l,c)==100,100,0),
        'BearishEngulfing': lambda o,h,l,c: np.where(talib.CDLENGULFING(o,h,l,c)==-100,-100,0),
        'PiercingLine': talib.CDLPIERCING,
        'DarkCloudCover': talib.CDLDARKCLOUDCOVER,
        'MorningStar': talib.CDLMORNINGSTAR,
        'EveningStar': talib.CDLEVENINGSTAR,
        'ThreeWhiteSoldiers': talib.CDL3WHITESOLDIERS,
        'ThreeBlackCrows': talib.CDL3BLACKCROWS,
        # ... 可根据需求增加更多形态
    }
    for name, fn in funcs.items():
        df[name] = fn(df['open'].values, df['high'].values, df['low'].values, df['close'].values)
    return df

def aggregate_candlestick_signals(df):
    bullish = ['Hammer','InvertedHammer','BullishEngulfing','PiercingLine','MorningStar','ThreeWhiteSoldiers']
    bearish = ['BearishEngulfing','DarkCloudCover','EveningStar','ThreeBlackCrows']
    df['bullish_score'] = df[bullish].eq(100).sum(axis=1)
    df['bearish_score'] = df[bearish].eq(-100).sum(axis=1)
    df['candlestick_score'] = df['bullish_score'] - df['bearish_score']
    return df

def integrate_candlestick_features(df):
    df = calculate_patterns(df)
    df = aggregate_candlestick_signals(df)
    return df.dropna()

# === 特征工程 ===
class AdvancedFeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, df):
        df = df.copy()
        if not isinstance(df.index, pd.DatetimeIndex):
            df.index = pd.to_datetime(df.index)
        df['price_change'] = df['close'].pct_change()
        df['high_low_ratio'] = (df['high'] - df['low']) / df['close']
        df['close_open_ratio'] = (df['close'] - df['open']) / df['open']
        df['rsi'] = talib.RSI(df['close'])
        df['macd'], df['macd_signal'], _ = talib.MACD(df['close'])
        df['volume_ma'] = df['volume'].rolling(20).mean()
        df['volume_ratio'] = df['volume'] / df['volume_ma']
        df['price_volume_corr'] = df['close'].rolling(20).corr(df['volume'])
        df['rsi_divergence'] = df['close'] - df['rsi']
        return df.dropna()

# === Enhanced MetaModel ===
class EnhancedMetaModel:
    def __init__(self, n_pca=20, l1_ratio=0.5):
        # 降维器
        self.pca = PCA(n_components=n_pca)
        # 基模型，增加正则化超参
        self.base_models = [
            ('xgb', XGBClassifier(
                n_estimators=100,
                reg_alpha=1.0,      # L1 正则系数
                reg_lambda=1.0,     # L2 正则系数
                use_label_encoder=False,
                eval_metric='mlogloss'
            )),
            ('rf', RandomForestClassifier(
                n_estimators=100,
                max_depth=8,
                class_weight='balanced'  # 同时保证样本不平衡时的权重
            )),
            ('gbm', GradientBoostingClassifier(
                n_estimators=100,
                learning_rate=0.1
            ))
        ]
        # 元模型：带弹性网（L1+L2）
        self.meta_model = Pipeline([
            ('scaler', StandardScaler()),
            ('pca', PCA(n_components=n_pca)),  # 再做一次降维
            ('clf', LogisticRegression(
                penalty='elasticnet',
                solver='saga',
                l1_ratio=l1_ratio,
                C=1.0,
                max_iter=1000,
                class_weight='balanced'
            ))
        ])
        self.scaler = StandardScaler()
        self.encoder = LabelEncoder()

    def fit(self, X, y):
        # 先标准化+PCA
        X_pca = self.pca.fit_transform(self.scaler.fit_transform(X))
        y_enc = self.encoder.fit_transform(y)

        # 训练基模型并收集它们的概率预测
        meta_X = []
        for name, model in self.base_models:
            model.fit(X_pca, y_enc)
            meta_X.append(model.predict_proba(X_pca))
        meta_X = np.hstack(meta_X)

        # 再用元模型做训练（管道里已经包含 scaler+pca）
        self.meta_model.fit(X, y_enc)  # 注意：管道内部会再做一次预处理
        return self

    def predict(self, X):
        # 基模型输出
        X_pca = self.pca.transform(self.scaler.transform(X))
        meta_X = np.hstack([m.predict_proba(X_pca) for _, m in self.base_models])
        # 元模型预测
        preds = self.meta_model.predict(X)
        return self.encoder.inverse_transform(preds)

# === 自定义 Block Bootstrap 白检验 P-value ===
def block_bootstrap_pval(returns, B=1000, block_len=5, seed=42):
    np.random.seed(seed)
    n = len(returns)
    indices = np.arange(n)
    boot_means = []
    for _ in range(B):
        sample_idx = []
        nb = int(np.ceil(n / block_len))
        for _ in range(nb):
            start = np.random.randint(0, n - block_len + 1)
            sample_idx.extend(indices[start: start + block_len])
        sample_idx = sample_idx[:n]
        boot_means.append(returns[sample_idx].mean())
    d_bar = returns.mean()
    return np.mean([d_bar <= m for m in boot_means])

# === 权重缩放评估函数 ===
def weighted_signal_evaluation(pred_list, y_list, sharpe_list):
    weights = np.clip(sharpe_list, 0, None)
    weights = weights / weights.sum() if weights.sum()>0 else np.ones_like(weights)/len(weights)
    combined = sum(w * (np.sign(y) * 0.001 * np.sign(p))
                   for w, p, y in zip(weights, pred_list, y_list))
    sharpe = combined.mean()/combined.std()*np.sqrt(252*24*4) if combined.std() else 0
    acc = accuracy_score(np.concatenate(y_list), np.concatenate(pred_list))
    return {'accuracy': acc, 'sharpe': sharpe}

# === 主流程 ===
def run_pipeline(csv_path):
    df = pd.read_csv(csv_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df.set_index('timestamp', inplace=True)
    df = df.resample('15min').agg({'open':'first','high':'max','low':'min','close':'last','volume':'sum'}).dropna()
    # df = df.iloc[-96*30:]

    # 1. 分位数标签 & 丢弃中性样本
    target_df = calculate_target_quantile(df, future_bars=3, lower_q=0.2, upper_q=0.8)
    df['direction'] = target_df['direction']
    df['magnitude'] = target_df['magnitude']
    df = df.dropna(subset=['direction'])
    print("方向标签分布:\n", df['direction'].value_counts(normalize=True))

    # 2. 特征工程 + 蜡烛图特征
    df_feat = AdvancedFeatureEngineer().fit_transform(df)
    df_feat = integrate_candlestick_features(df_feat)

    X = df_feat.drop(columns=['open','high','low','close','volume','direction','magnitude'])
    y = df_feat['direction']

    results, pred_list, y_list, sharpe_list = [], [], [], []
    train_len = 5000   # 训练集用 5000 根 15min K 线
    test_len  = 1000   # 测试集用 1000 根 15min K 线
    step      = 1000   # 每次向前滑动 1000 根

    n = len(X)
    for start in range(0, n - train_len - test_len + 1, step):
        train_idx = range(start, start + train_len)
        test_idx  = range(start + train_len, start + train_len + test_len)

        X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
        X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]

        if y_train.nunique()<2:
            print(f"Window {start}-{start+train_len+test_len} skip: only {y_train.unique()}")
            continue

        # SMOTE 过采样
        X_res, y_res = SMOTE().fit_resample(X_train, y_train)
        model = EnhancedMetaModel().fit(X_res, y_res)
        preds = model.predict(X_test)

        # 计算回报 & Sharpe
        ret = df['close'].iloc[test_idx].pct_change().shift(-1) * np.sign(preds)
        sharpe = ret.mean()/ret.std()*np.sqrt(252*24*4) if ret.std() else 0
        sharpe_list.append(sharpe)

        # 输出报告
        print(f"\n======= Window {start}-{start+train_len-1} → Test {start+train_len}-{start+train_len+test_len-1} =======")
        print(classification_report(y_test, preds, digits=3))
        print(f"Sharpe: {sharpe:.2f} | Acc: {accuracy_score(y_test, preds):.4f}")

        results.append({'accuracy': accuracy_score(y_test, preds), 'sharpe': sharpe})
        pred_list.append(preds)
        y_list.append(y_test.values)

    if not results:
        print("❌ no valid folds")
        return

    print("\n======= Summary =======")
    for i,r in enumerate(results,1): print(f"Fold {i}: Acc={r['accuracy']:.4f}, Sharpe={r['sharpe']:.2f}")

    # 白检验
    all_ret = np.concatenate([np.sign(y)*0.001*np.sign(p) for p,y in zip(pred_list,y_list)])
    print("White Reality Check p-value:", block_bootstrap_pval(all_ret))

    # 权重缩放评估
    w = weighted_signal_evaluation(pred_list, y_list, np.array(sharpe_list))
    print(f"Weighted Acc={w['accuracy']:.4f}, Sharpe={w['sharpe']:.2f}")

    # 保存最终模型
    joblib.dump(EnhancedMetaModel().fit(X,y), "final_model.pkl")
    print("Saved final_model.pkl")

    # 1. 生成特征 + 蜡烛图形态
    df_feat = AdvancedFeatureEngineer().fit_transform(df)
    df_feat = integrate_candlestick_features(df_feat)

    # 2. 拷贝出来用于可视化
    df_plot = df_feat.copy()

    # 3. 计算均线和 RSI
    df_plot['MA20'] = df_plot['close'].rolling(20).mean()
    df_plot['MA50'] = df_plot['close'].rolling(50).mean()
    df_plot['RSI']  = talib.RSI(df_plot['close'])

    # 4. 定义你要画的形态列表
    patterns = ['ThreeLineStrike', 'InvertedHammer', 'HangingMan']

    # 5. 循环生成 Signal_ 列
    for pat in patterns:
        if pat not in df_plot.columns:
            print(f"⚠️ pattern '{pat}' not found, skipping")
            continue
        sig_col = f'Signal_{pat}'
        df_plot[sig_col] = np.where(
            df_plot[pat] == 100,  1,
            np.where(df_plot[pat] == -100, -1, 9)
        )

    # 6. 最后调用可视化函数
    plot_pattern_results(
        df = df_plot,
        patterns = patterns,
        symbol = 'BTCUSDT',
        max_points = 2000,
        buffer     = 50,
        open_browser = True
    )



In [ ]:
csv_path = "../data/BTCUSDT_1min_2024-05-01_to_2025-05-01.csv"
run_pipeline(csv_path)

方向标签分布:
 direction
 0    0.600023
-1    0.199989
 1    0.199989
Name: proportion, dtype: float64

======= Window 0-4999 → Test 5000-5999 =======
              precision    recall  f1-score   support

          -1      0.396     0.128     0.194       148
           0      0.818     0.762     0.789       731
           1      0.210     0.471     0.291       121

    accuracy                          0.633      1000
   macro avg      0.475     0.454     0.425      1000
weighted avg      0.682     0.633     0.641      1000

Sharpe: 0.10 | Acc: 0.6330

======= Window 1000-5999 → Test 6000-6999 =======
              precision    recall  f1-score   support

          -1      0.308     0.108     0.160       259
           0      0.608     0.378     0.466       513
           1      0.268     0.693     0.386       228

    accuracy                          0.380      1000
   macro avg      0.395     0.393     0.338      1000
weighted avg      0.453     0.380     0.369      1000

Sharpe: 2.20 | 

In [ ]:
def fetch_live_klines(symbol='BTCUSDT', interval='15m', limit=5000):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        'open_time','open','high','low','close','volume',
        'close_time','quote_asset_volume','trades',
        'taker_base','taker_quote','ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['open_time'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df[['open','high','low','close','volume']].astype(float)

# ===================== 主流程入口 =====================
def run_pipeline(symbol='BTCUSDT',
                 csv_path=None,
                 live=False,
                 interval='15m',
                 history_limit=5000):
    # 1. 数据加载
    if live:
        # 从 Testnet 拉取最近 history_limit 根 K 线
        df = fetch_live_klines(symbol=symbol,
                               interval=interval,
                               limit=history_limit)
    else:
        # 从 CSV 读历史
        df = pd.read_csv(csv_path, parse_dates=['timestamp'])
        df.set_index('timestamp', inplace=True)
    
    # 2. 统一重采样到 interval
    df = df.resample(interval).agg({
        'open': 'first',
        'high': 'max',
        'low':  'min',
        'close':'last',
        'volume':'sum'
    }).dropna()
    
    # —— 下面开始跟你原来一样的 pipeline —— #
    
    # 3. 分位数标签
    target_df = calculate_target_quantile(df, future_bars=3, lower_q=0.2, upper_q=0.8)
    df['direction'] = target_df['direction']
    df['magnitude'] = target_df['magnitude']
    df.dropna(subset=['direction'], inplace=True)
    
    # 4. 特征工程 + 蜡烛图特征
    df_feat = AdvancedFeatureEngineer().fit_transform(df)
    df_feat = integrate_candlestick_features(df_feat)
    
    X = df_feat.drop(columns=['open','high','low','close','volume','direction','magnitude'])
    y = df_feat['direction']
    
    # 5. 滚动窗口回测
    results, pred_list, y_list, sharpe_list = [], [], [], []
    train_len, test_len, step = 5000, 1000, 1000
    n = len(X)
    for start in range(0, n-train_len-test_len+1, step):
        # 5.1 划分
        tr_idx = range(start, start+train_len)
        te_idx = range(start+train_len, start+train_len+test_len)
        X_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        X_te, y_te = X.iloc[te_idx], y.iloc[te_idx]
        if y_tr.nunique()<2:
            continue
        # 5.2 过采样、训练、预测
        X_res, y_res = SMOTE().fit_resample(X_tr, y_tr)
        model = EnhancedMetaModel().fit(X_res, y_res)
        preds = model.predict(X_te)
        # 5.3 计算 Sharpe/Acc
        ret = df['close'].iloc[te_idx].pct_change().shift(-1) * np.sign(preds)
        sharpe = ret.mean()/ret.std()*np.sqrt(252*24*4) if ret.std() else 0
        sharpe_list.append(sharpe)
        results.append({'accuracy':accuracy_score(y_te,preds),
                        'sharpe':sharpe})
        pred_list.append(preds); y_list.append(y_te.values)
        print(f"Fold@{start}: Acc={results[-1]['accuracy']:.3f} | Sharpe={sharpe:.2f}")
    
    # 6. 汇总、白检验、权重评估、保存模型（同前）…
    # … 你的 Summary/WhiteRC/weighted evaluation …
    final_model = EnhancedMetaModel().fit(X,y)
    joblib.dump(final_model, 'final_model.pkl')

    # 1. 生成特征 + 蜡烛图形态
    df_feat = AdvancedFeatureEngineer().fit_transform(df)
    df_feat = integrate_candlestick_features(df_feat)

    # 2. 拷贝出来用于可视化
    df_plot = df_feat.copy()

    # 3. 计算均线和 RSI
    df_plot['MA20'] = df_plot['close'].rolling(20).mean()
    df_plot['MA50'] = df_plot['close'].rolling(50).mean()
    df_plot['RSI']  = talib.RSI(df_plot['close'])

    # 4. 定义你要画的形态列表
    patterns = ['ThreeLineStrike', 'InvertedHammer', 'HangingMan']

    # 5. 循环生成 Signal_ 列
    for pat in patterns:
        if pat not in df_plot.columns:
            print(f"pattern '{pat}' not found, skipping")
            continue
        sig_col = f'Signal_{pat}'
        df_plot[sig_col] = np.where(
            df_plot[pat] == 100,  1,
            np.where(df_plot[pat] == -100, -1, 9)
        )

    # 6. 最后调用可视化函数
    plot_pattern_results(
        df = df_plot,
        patterns = patterns,
        symbol = 'BTCUSDT',
        max_points = 2000,
        buffer     = 50,
        open_browser = True
    )


In [ ]:
import time
import os
import pandas as pd
import numpy as np
from binance.client import Client
import joblib
import talib

from prediction_logger import PredictionLogger
from signal_logger import SignalHistoryLogger

# —— Step 0: 初始化 Testnet 客户端 & 日志器 & 离线加载模型 —— #
from binance.client import Client
# sys.path.append(os.path.abspath(".."))  # root /PycharmProjects/MMAT
from config.load_env import load_keys

keys = load_keys()
#print("Loaded keys:", keys)
client = Client(keys['api_key'], keys['secret_key'])

# 载入提前离线训练并保存好的模型
model = joblib.load("final_model.pkl")

pred_logger = PredictionLogger()
sig_logger  = SignalHistoryLogger("signal_history.csv")

# 确保程序退出时会把日志落盘
import atexit
atexit.register(lambda: pred_logger.save_to_csv("prediction_log.csv"))
atexit.register(lambda: sig_logger.save_to_csv())

# —— Step 1: 实时拉取 OHLCV —— #
def fetch_ohlcv(symbol="BTCUSDT", interval="15m", limit=500):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        'open_time','open','high','low','close','volume',
        'close_time','quote_vol','trades','tb_base','tb_quote','ignore'])
    df['timestamp'] = pd.to_datetime(df['open_time'], unit='ms')
    df.set_index('timestamp', inplace=True)
    return df[['open','high','low','close','volume']].astype(float)

# —— Step 2: 画图函数（复用你已有的 plot_pattern_results） —— #
# from your_viz_module import plot_pattern_results

# —— Step 3: 实时主循环 —— #
def live_loop(symbol="BTCUSDT",
              interval="15m",
              history_bars=500):
    last_close = None

    while True:
        try:
            # 3.1 获取最新数据
            df = fetch_ohlcv(symbol, interval, history_bars)
            
            # 3.2 特征工程（只需工程出模型需要的 X）
            df_feat = AdvancedFeatureEngineer().fit_transform(df)
            df_feat = integrate_candlestick_features(df_feat)
            X_live = df_feat.drop(columns=['open','high','low','close','volume','direction','magnitude'],
                                  errors='ignore')
            
            # 3.3 对最新一根bar 做预测
            x0 = X_live.iloc[[-1]]                 # shape (1, n_features)
            pred = model.predict(x0)[0]            # -1 / 0 / +1
            
            # 3.4 记录预测 & 实际命中
            now = X_live.index[-1]
            close_now  = df['close'].iloc[-1]
            if last_close is not None:
                # UP / DOWN / HOLD
                label = "UP" if pred==1 else ("DOWN" if pred==-1 else "HOLD")
                pred_logger.record_prediction(now, label, close_now, last_close)
            
            # 3.5 如果是真实信号（非 0），落地到 signal_history
            if pred == 1:
                sig_logger.add_signal('bullish', now, close_now, trigger=f"pred==1")
            elif pred == -1:
                sig_logger.add_signal('bearish', now, close_now, trigger=f"pred==-1")
            
            last_close = close_now
            
            # 3.6 可视化最近信号（可选）
            df_plot = df_feat.copy()
            # 计算辅助指标
            df_plot['MA20'] = df_plot['close'].rolling(20).mean()
            df_plot['MA50'] = df_plot['close'].rolling(50).mean()
            df_plot['RSI']  = talib.RSI(df_plot['close'])
            # 生成 Signal_ 列
            patterns = ['ThreeLineStrike','InvertedHammer','HangingMan']
            for pat in patterns:
                if pat in df_plot:
                    df_plot[f"Signal_{pat}"] = np.where(
                        df_plot[pat]==100, 1,
                        np.where(df_plot[pat]==-100, -1, 9)
                    )
            # 调用你写好的画图函数，只看最后 200 根 K 线
            plot_pattern_results(
                df=df_plot,
                patterns=patterns,
                symbol=symbol,
                max_points=200,
                buffer=20,
                open_browser=False  # 如果太频繁就关掉自动打开
            )
            
            print(f"[{now}] Pred={pred}, HitRate={pred_logger.get_hit_rate():.3f}")
            time.sleep(10)  # 每 5 分钟跑一次

        except Exception as e:
            print(f"[LiveLoop] Error: {e}, retry in 60s")
            time.sleep(60)

if __name__=="__main__":
    live_loop(symbol="BTCUSDT", interval="15m", history_bars=500)


[LiveLoop] Error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- adx
- atr
- cci
- day_of_week
- ema_15
- ...
Feature names seen at fit time, yet now missing:
- price_volume_corr
- rsi_divergence
, retry in 60s
[LiveLoop] Error: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- adx
- atr
- cci
- day_of_week
- ema_15
- ...
Feature names seen at fit time, yet now missing:
- price_volume_corr
- rsi_divergence
, retry in 60s


KeyboardInterrupt: 